In [81]:
import os
import numpy as np
import joblib
import plotly.graph_objs as go
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Masking, LSTM, Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import LogCosh
from tensorflow.keras.optimizers import Adam

In [2]:
# TODO: specify path to the folder containing the features data
directory = '../Data/Features Data'

In [3]:
# these are the folders for participants who did not use the 3 kg weights
skip = {
    'individual_00' : ['experiment_1', 'experiment_2'],
    'individual_03' : ['experiment_1', 'experiment_2'],
    'individual_04' : ['experiment_1', 'experiment_2'],
    'individual_07' : ['experiment_1', 'experiment_4'],
    'individual_08' : ['experiment_1', 'experiment_2'],
    'individual_11' : ['experiment_2'],
    'individual_14' : ['experiment_2'],
    'individual_17' : ['experiment_1'],
    'individual_23' : ['experiment_3', 'experiment_4'],
    'individual_47' : ['experiment_1', 'experiment_2'],
    'individual_49' : ['experiment_1', 'experiment_2'],
    'individual_51' : ['experiment_1'],
    'individual_52' : ['experiment_1', 'experiment_2'],
    'individual_53' : ['experiment_1'],
    'individual_54' : ['experiment_1', 'experiment_2'],
    'individual_55' : ['experiment_1'],
}
skip_folders = []
for individual_folder_skip, experiments_skip in skip.items():
  for experiment_folder_skip in experiments_skip:
    skip_folders.append(f'{directory}/{individual_folder_skip}/{experiment_folder_skip}')

In [4]:
train_participants = ['individual_00','individual_01','individual_02','individual_05','individual_06','individual_07','individual_08','individual_10','individual_11','individual_12','individual_13','individual_15','individual_16','individual_18','individual_19','individual_20','individual_21','individual_22','individual_23','individual_24','individual_25','individual_26','individual_27','individual_28','individual_29','individual_31','individual_32','individual_33','individual_34','individual_36','individual_37','individual_39','individual_40','individual_42','individual_44','individual_45','individual_48','individual_50','individual_56','individual_57','individual_58','individual_60','individual_61','individual_62','individual_63','individual_64','individual_66']
test_participants = ['individual_14','individual_65','individual_38','individual_59','individual_43','individual_46','individual_35','individual_09','individual_41','individual_30']

In [5]:
# load training data
all_X, all_y = [], []
for individual_folder in train_participants:
  individual_X, individual_y = [], []
  experiment_folders = [folder for folder in sorted(os.listdir(f'{directory}/{individual_folder}')) if 'experiment' in folder]
  for i in range(len(experiment_folders)):
      experiment_folder = experiment_folders[i]
      folder_path = f'{directory}/{individual_folder}/{experiment_folder}'

      if folder_path not in skip_folders:
        experiment_X = np.load(f'{folder_path}/X.npy', allow_pickle=True)
        experiment_y = np.load(f'{folder_path}/y.npy', allow_pickle=True)

        # only keep participant data up to level 7 fatigue
        threshold = 7
        mask = experiment_y >= threshold
        index = np.argmax(mask)

        if len(experiment_y[:index]) > 0:
            individual_X.append(experiment_X[:index])
            individual_y.append(experiment_y[:index])
        # skips individual_13 experiment_1 because this experiment never reached level 7 fatigue

  all_X.append(individual_X)
  all_y.append(individual_y)

In [6]:
# load testing data
test_X, test_y = [], []
for individual_folder in test_participants:
  individual_X, individual_y = [], []
  experiment_folders = [folder for folder in sorted(os.listdir(f'{directory}/{individual_folder}')) if 'experiment' in folder]
  for i in range(len(experiment_folders)):
      experiment_folder = experiment_folders[i]
      folder_path = f'{directory}/{individual_folder}/{experiment_folder}'

      if folder_path not in skip_folders:
        experiment_X = np.load(f'{folder_path}/X.npy', allow_pickle=True)
        experiment_y = np.load(f'{folder_path}/y.npy', allow_pickle=True)

        # only keep participant data up to level 7 fatigue
        threshold = 7
        mask = experiment_y >= threshold
        index = np.argmax(mask)

        if len(experiment_y[:index]) > 0:
            individual_X.append(experiment_X[:index])
            individual_y.append(experiment_y[:index])

  test_X.append(individual_X)
  test_y.append(individual_y)

In [7]:
def get_data(val_individuals):
  """ 
  Split the training data into train and validation sets, given the indices of the validation individuals whose data to withold from training
  """
  X_val, y_val = [], []
  for i in val_individuals:
    num_experiments = len(all_X[i])
    for experiment in range(num_experiments):
      X_val.append(all_X[i][experiment])
      y_val.append(all_y[i][experiment])

  X_train, y_train = [], []
  for i in range(len(all_X)):
    if i not in val_individuals:
      num_experiments = len(all_X[i])
      for experiment in range(num_experiments):
        X_train.append(all_X[i][experiment])
        y_train.append(all_y[i][experiment])

  # standardize features using training data
  X_train_flattened = np.concatenate(X_train, axis=0)
  scaler = StandardScaler()
  scaler.fit(X_train_flattened)
  X_train = [scaler.transform(X_train_i) for X_train_i in X_train]
  X_val = [scaler.transform(X_val_i) for X_val_i in X_val]

  return X_val, y_val, X_train, y_train, scaler

In [8]:
# computes the longest sequence in all of the data to use for RNN padding
max_seq = max([max([y_attempt.shape[0] for y_attempt in y_individual]) for y_individual in all_y if len(y_individual) > 0])

In [29]:
#TODO specify path to store the best model checkpoint 
model_folder = '../System/models' 

In [56]:
def train_rnn(X_train, y_train, X_val, y_val, indices_to_try=None):
    train_data_padded = pad_sequences(X_train, maxlen=max_seq, padding='post', dtype='float32', value=-1)
    train_labels_padded = pad_sequences(y_train, maxlen=max_seq, padding='post', dtype='float32', value=-1)
    train_labels_padded = np.expand_dims(train_labels_padded, axis=-1)

    val_data_padded = pad_sequences(X_val, maxlen=max_seq, padding='post', dtype='float32', value=-1)
    val_labels_padded = pad_sequences(y_val, maxlen=max_seq, padding='post', dtype='float32', value=-1)
    val_labels_padded = np.expand_dims(val_labels_padded, axis=-1)

    if indices_to_try:
        train_data_padded = train_data_padded[:, :, indices_to_try]
        val_data_padded = val_data_padded[:, :, indices_to_try]

    num_experiments, num_reps, num_features = train_data_padded.shape
    rnn_model = Sequential()
    rnn_model.add(Input(shape=(num_reps, num_features)))
    rnn_model.add(Masking(mask_value=-1))
    rnn_model.add(LSTM(64, return_sequences=True))
    rnn_model.add(Dense(1))

    checkpoint = ModelCheckpoint(
        filepath=f'{model_folder}/temp.keras',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=0
    )

    optimizer_stage1 = Adam(learning_rate=0.001)
    rnn_model.compile(optimizer=optimizer_stage1, loss=LogCosh())

    rnn_model.fit(train_data_padded, train_labels_padded,
                  epochs=200, batch_size=128,
                  validation_data=(val_data_padded, val_labels_padded),
                  callbacks=[checkpoint],
                  verbose=0
                  )

    optimizer_stage2 = Adam(learning_rate=0.0001)
    rnn_model.compile(optimizer=optimizer_stage2, loss=LogCosh())

    rnn_model.fit(train_data_padded, train_labels_padded,
                  epochs=200, batch_size=128,
                  validation_data=(val_data_padded, val_labels_padded),
                  callbacks=[checkpoint],
                  verbose=0
                  )

    return rnn_model

In [11]:
def predict_rnn(model, X_test, indices_to_try=None):
  if indices_to_try:
    X_test = X_test[:,indices_to_try]
  return np.squeeze(model.predict(np.expand_dims(X_test, axis=0), verbose=0))

In [12]:
def get_mse(prediction, actual):
  return np.mean((prediction - actual) ** 2)

In [13]:
# get all individuals with nonzero experiments
all_individuals = []
for individual_to_withold in range(len(all_X)):
  if len(all_X[individual_to_withold]) > 0:
    all_individuals.append(individual_to_withold)

In [14]:
selected_features = [9, 12, 13, 18, 32, 73, 75, 114, 118, 156, 205, 225, 262]

In [74]:
# randomly select 6 individuals to use for validation data and train the model
individuals_to_withold = np.random.choice(all_individuals, size=6, replace=False)
X_val, y_val, X_train, y_train, scaler = get_data(individuals_to_withold)      

In [76]:
rnn_model = train_rnn(X_train, y_train, X_val, y_val, indices_to_try = selected_features)
rnn_model = load_model(f'{model_folder}/temp.keras') # load the best checkpoint during training

In [97]:
mses = []
for idx in range(len(test_participants)):
    print(f'Testing on {test_participants[idx]}')
    individual_X = test_X[idx]
    individual_y = test_y[idx]
    for exp in range(len(individual_X)):
        experiment_X = scaler.transform(individual_X[exp])
        experiment_y = individual_y[exp]
        prediction = predict_rnn(rnn_model, experiment_X, indices_to_try=selected_features)
        mse = get_mse(prediction, experiment_y)
        print(mse)
        mses.append(mse)
print(f'Average MSE: {np.mean(mses)} +/- {np.std(mses)}')

Testing on individual_14
0.30507176029393784
Testing on individual_65
0.6102292011527415
1.0258161071973668
Testing on individual_38
2.838266464001016
Testing on individual_59
0.6015139286239303
Testing on individual_43
2.746992035732614
0.6885119368944095
Testing on individual_46
3.8382308267256344
0.5512005789783218
Testing on individual_35
1.2544289842440026
4.9431094441927454
Testing on individual_09
2.291792558364928
0.4038220914659495
Testing on individual_41
0.5478056225720518
3.8389211916659316
Testing on individual_30
0.2598750385897005
Average MSE: 1.671599235668455 +/- 1.475032222133471


In [78]:
# save the model and the scaler to apply for real-time use
rnn_model.save(f'{model_folder}/rnn.keras')
joblib.dump(scaler, f'{model_folder}/scaler.pkl')

['models/scaler.pkl']

In [86]:
idx = 1
individual_X = test_X[idx]
individual_y = test_y[idx]
exp = 0
experiment_X = scaler.transform(individual_X[exp])
experiment_y = individual_y[exp]
prediction = predict_rnn(rnn_model, experiment_X, indices_to_try=selected_features)
mse = get_mse(prediction, experiment_y)
print(mse)

0.6102292011527415


In [90]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(prediction))), y=prediction, name='Prediction'))
fig.add_trace(go.Scatter(x=list(range(len(experiment_y))), y=experiment_y, name='Actual'))
fig.update_layout(width=600)
fig.show()